## CRF
CRF는 Conditional Random Field의 약자로 양방향 LSTM을 위해 탄생한 모델이 아
니라 이전에 독자적으로 존재해왔던 모델입니다. 이를 양방향 LSTM 모델 위에 하나의
층으로 추가하여, 양방향 LSTM + CRF 모델이 탄생하였습니다. 여기서는 CRF의 수식적
 이해가 아니라 양방향 LSTM + CRF 모델의 직관에 대해서 이해합니다.

In [ ]:
!pip install keras-crf